#### **Extracción de petróleo OilyGiant**

Trabajas en la compañía de extracción de petróleo OilyGiant. Tu tarea es encontrar los mejores lugares donde abrir 200 pozos nuevos de petróleo.
##### Para completar esta tarea, tendrás que realizar los siguientes pasos:
- Leer los archivos con los parámetros recogidos de pozos petrolíferos en la región seleccionada: calidad de crudo y volumen de reservas.
- Crear un modelo para predecir el volumen de reservas en pozos nuevos.
- Elegir los pozos petrolíferos que tienen los valores estimados más altos.
- Elegir la región con el beneficio total más alto para los pozos petrolíferos seleccionados.
- Tienes datos sobre muestras de crudo de tres regiones. Ya se conocen los parámetros de cada pozo petrolero de la región. Crea un modelo que ayude a elegir la región con el mayor margen de beneficio. Analiza los beneficios y riesgos potenciales utilizando la técnica bootstrapping.
##### Condiciones:
- Solo se debe usar la regresión lineal para el entrenamiento del modelo.
- Al explorar la región, se lleva a cabo un estudio de 500 puntos con la selección de los mejores 200 puntos para el cálculo del beneficio.
- El presupuesto para el desarrollo de 200 pozos petroleros es de 100 millones de dólares.
- Un barril de materias primas genera 4.5 USD de ingresos. El ingreso de una unidad de producto es de 4500 dólares (el volumen de reservas está expresado en miles de barriles).
- Después de la evaluación de riesgo, mantén solo las regiones con riesgo de pérdidas inferior al 2.5%. De las que se ajustan a los criterios, se debe seleccionar la región con el beneficio promedio más alto.
##### Descripción de datos
- id: identificador único de pozo de petróleo
- f0, f1, f2: tres características de los puntos (su significado específico no es importante, pero las características en sí son significativas)
- product: volumen de reservas en el pozo de petróleo (miles de barriles).

In [2]:
#Importar librerías
import pandas as pd
import numpy as np
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.linear_model import LinearRegression

In [5]:
df_0 = pd.read_csv('geo_data_0.csv')
df_1 = pd.read_csv('geo_data_1.csv')
df_2 = pd.read_csv('geo_data_2.csv')

#Explorar los datos
print(df_0.head())
print(df_1.head())
print(df_2.head())

      id        f0        f1        f2     product
0  txEyH  0.705745 -0.497823  1.221170  105.280062
1  2acmU  1.334711 -0.340164  4.365080   73.037750
2  409Wp  1.022732  0.151990  1.419926   85.265647
3  iJLyR -0.032172  0.139033  2.978566  168.620776
4  Xdl7t  1.988431  0.155413  4.751769  154.036647
      id         f0         f1        f2     product
0  kBEdx -15.001348  -8.276000 -0.005876    3.179103
1  62mP7  14.272088  -3.475083  0.999183   26.953261
2  vyE1P   6.263187  -5.948386  5.001160  134.766305
3  KcrkZ -13.081196 -11.506057  4.999415  137.945408
4  AHL4O  12.702195  -8.147433  5.004363  134.766305
      id        f0        f1        f2     product
0  fwXo0 -1.146987  0.963328 -0.828965   27.758673
1  WJtFt  0.262778  0.269839 -2.530187   56.069697
2  ovLUW  0.194587  0.289035 -5.586433   62.871910
3  q6cA6  2.236060 -0.553760  0.930038  114.572842
4  WPMUX -0.515993  1.716266  5.899011  149.600746


In [13]:
#Información de los datos
print(df_0.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 5 columns):
 #   Column   Non-Null Count   Dtype  
---  ------   --------------   -----  
 0   id       100000 non-null  object 
 1   f0       100000 non-null  float64
 2   f1       100000 non-null  float64
 3   f2       100000 non-null  float64
 4   product  100000 non-null  float64
dtypes: float64(4), object(1)
memory usage: 3.8+ MB
None


In [14]:
print(df_1.info())
print(df_2.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 5 columns):
 #   Column   Non-Null Count   Dtype  
---  ------   --------------   -----  
 0   id       100000 non-null  object 
 1   f0       100000 non-null  float64
 2   f1       100000 non-null  float64
 3   f2       100000 non-null  float64
 4   product  100000 non-null  float64
dtypes: float64(4), object(1)
memory usage: 3.8+ MB
None
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 5 columns):
 #   Column   Non-Null Count   Dtype  
---  ------   --------------   -----  
 0   id       100000 non-null  object 
 1   f0       100000 non-null  float64
 2   f1       100000 non-null  float64
 3   f2       100000 non-null  float64
 4   product  100000 non-null  float64
dtypes: float64(4), object(1)
memory usage: 3.8+ MB
None


In [15]:
# Divide los datos en un conjunto de entrenamiento y un conjunto de validación en una proporción de 75:25
def preparar_datos(df, features, target, test_size=0.25, random_state=42):
    x_train, x_valid, y_train, y_valid = train_test_split(df[features], df[target],test_size=test_size, random_state=random_state)
    return x_train, x_valid, y_train, y_valid
#Definir las características y la variable objetivo
features = ['f0', 'f1', 'f2']
target = 'product'

In [16]:
# Entrena el modelo y haz predicciones para el conjunto de validación.
def train_predict(x_train,y_train,x_valid, y_valid):
    model = LinearRegression()
    model.fit(x_train,y_train)
    y_pred = model.predict(x_valid)
    mse = mean_squared_error(y_valid, y_pred)
    print(f'Mean Squared Error: {mse}')
    return model, y_pred

In [17]:
# Para geo_data_0.csv
x_train_0, x_valid_0, y_train_0, y_valid_0 = preparar_datos(df_0, features, target)
model_0, preds_0 = train_predict(x_train_0, y_train_0, x_valid_0, y_valid_0)
print(f'Predicciones para geo_data_0: {preds_0[:5]}')


Mean Squared Error: 1425.5608700093812
Predicciones para geo_data_0: [101.90101715  78.21777385 115.26690103 105.61861791  97.9801849 ]


In [18]:
# Para geo_data_1.csv
x_train_1, x_valid_1, y_train_1, y_valid_1 = preparar_datos(df_1, features, target)
model_1, preds_1 = train_predict(x_train_1, y_train_1, x_valid_1, y_valid_1)
print(f'Predicciones para geo_data_1: {preds_1[:5]}')

Mean Squared Error: 0.7925986566392037
Predicciones para geo_data_1: [ 8.44738063e-01  5.29216119e+01  1.35110385e+02  1.09494863e+02
 -4.72915824e-02]


In [19]:
# Para geo_data_2.csv
x_train_2, x_valid_2, y_train_2, y_valid_2 = preparar_datos(df_2, features, target)
model_2, preds_2 = train_predict(x_train_2, y_train_2, x_valid_2, y_valid_2)
print(f'Predicciones para geo_data_2: {preds_2[:5]}')

Mean Squared Error: 1611.6910636385903
Predicciones para geo_data_2: [ 98.30191642 101.59246124  52.4490989  109.92212707  72.41184733]


#### Interpretación:
geo_data_1 es el más predecible, el modelo logra aprender bien la relación entre las variables f0, f1, f2 y el product. Esto puede deberse a que las variables están más correlacionadas con el objetivo o tienen una distribución más adecuada.

geo_data_0 y geo_data_2 tienen ruido o relaciones no lineales, el alto MSE sugiere que el modelo no logra capturar bien la relación, quizá por la naturaleza de los datos o por una mala elección del modelo.

Prepárate para el cálculo de ganancias:
Almacena todos los valores necesarios para los cálculos en variables separadas.
Dada la inversión de 100 millones por 200 pozos petrolíferos, de media un pozo petrolífero debe producir al menos un valor de 500,000 dólares en unidades para evitar pérdidas (esto es equivalente a 111.1 unidades). Compara esta cantidad con la cantidad media de reservas en cada región.
Presenta conclusiones sobre cómo preparar el paso para calcular el beneficio.

In [21]:
# Cálculo del umbral mínimo de producción por pozo para no tener pérdidas
umbral_unidades = presupuesto / num_pozos / precio_unidad  # 100_000_000 / 200 / 4500 = 111.11 unidades

# Calcular la media de reservas por pozo en cada región
media_reservas_0 = df_0['product'].mean()
media_reservas_1 = df_1['product'].mean()
media_reservas_2 = df_2['product'].mean()

print(f"Umbral mínimo de producción por pozo para no tener pérdidas: {umbral_unidades:.2f} unidades")
print(f"Media de reservas por pozo en Región 0: {media_reservas_0:.2f} unidades")
print(f"Media de reservas por pozo en Región 1: {media_reservas_1:.2f} unidades")
print(f"Media de reservas por pozo en Región 2: {media_reservas_2:.2f} unidades")

# Conclusión
if (media_reservas_0 > umbral_unidades) and (media_reservas_1 > umbral_unidades) and (media_reservas_2 > umbral_unidades):
    print("Todas las regiones superan el umbral mínimo de reservas promedio por pozo. Se puede proceder al cálculo de beneficios para seleccionar la mejor región.")
else:
    print("Alguna región no supera el umbral mínimo de reservas promedio por pozo. Es necesario analizar el riesgo antes de invertir.")

Umbral mínimo de producción por pozo para no tener pérdidas: 111.11 unidades
Media de reservas por pozo en Región 0: 92.50 unidades
Media de reservas por pozo en Región 1: 68.83 unidades
Media de reservas por pozo en Región 2: 95.00 unidades
Alguna región no supera el umbral mínimo de reservas promedio por pozo. Es necesario analizar el riesgo antes de invertir.


#### Interpretación
geo_data_1 es el modelo más preciso, pero menor potencial económico, el modelo predice con alta precisión (MSE muy bajo).Sin embargo, la producción media por pozo es baja (68.83).No alcanza el umbral mínimo (111.11 unidades) por lo que verificamos que se tiene un riesgo alto de pérdida, a pesar de la certeza del modelo.

Escribe una función para calcular la ganancia de un conjunto de pozos de petróleo seleccionados y modela las predicciones:
Elige los 200 pozos con los valores de predicción más altos de cada una de las 3 regiones (es decir, archivos 'csv').
Resume el volumen objetivo de reservas según dichas predicciones. Almacena las predicciones para los 200 pozos para cada una de las 3 regiones.
Calcula la ganancia potencial de los 200 pozos principales por región. Presenta tus conclusiones: propón una región para el desarrollo de pozos petrolíferos y justifica tu elección.

In [22]:
# Función para calcular la ganancia potencial basada en las predicciones del modelo
def calcular_ganancia(predicciones, num_pozos, precio_unidad, presupuesto):
    # Selecciona las 200 predicciones más altas
    top_preds = np.sort(predicciones)[-num_pozos:]
    # Suma el volumen estimado de reservas
    suma_reservas = top_preds.sum()
    # Calcula la ganancia potencial
    ganancia = suma_reservas * precio_unidad - presupuesto
    return ganancia, top_preds, suma_reservas

# Calcula la ganancia para cada región usando las predicciones del modelo
ganancia_0, top_preds_0, suma_reservas_0 = calcular_ganancia(preds_0, num_pozos, precio_unidad, presupuesto)
ganancia_1, top_preds_1, suma_reservas_1 = calcular_ganancia(preds_1, num_pozos, precio_unidad, presupuesto)
ganancia_2, top_preds_2, suma_reservas_2 = calcular_ganancia(preds_2, num_pozos, precio_unidad, presupuesto)

print(f"Región 0: Ganancia potencial = ${ganancia_0:,.0f}, Volumen total estimado = {suma_reservas_0:.2f} unidades")
print(f"Región 1: Ganancia potencial = ${ganancia_1:,.0f}, Volumen total estimado = {suma_reservas_1:.2f} unidades")
print(f"Región 2: Ganancia potencial = ${ganancia_2:,.0f}, Volumen total estimado = {suma_reservas_2:.2f} unidades")

# Conclusión: Selecciona la mejor región
ganancias = [ganancia_0, ganancia_1, ganancia_2]
mejor_region = np.argmax(ganancias)
print(f"\nLa mejor región para el desarrollo de pozos petrolíferos según las predicciones del modelo es la Región {mejor_region} con una ganancia potencial de ${ganancias[mejor_region]:,.0f}.")

Región 0: Ganancia potencial = $38,966,585, Volumen total estimado = 30881.46 unidades
Región 1: Ganancia potencial = $24,869,381, Volumen total estimado = 27748.75 unidades
Región 2: Ganancia potencial = $33,779,815, Volumen total estimado = 29728.85 unidades

La mejor región para el desarrollo de pozos petrolíferos según las predicciones del modelo es la Región 0 con una ganancia potencial de $38,966,585.


#### Interpretación
La región 0 destaca como la mejor opción a pesar de que: el modelo tiene mayor error de predicción (MSE alto) y la producción promedio está por debajo del umbral de rentabilidad.

Sin embargo, el modelo genera la mayor ganancia potencial absoluta ($38.97 millones), lo que sugiere que algunos pozos en Región 0 tienen producciones muy altas, compensando los menos productivos.

Calcula riesgos y ganancias para cada región:
Utilizando las predicciones que almacenaste en el paso 4.2, emplea la técnica del bootstrapping con 1000 muestras para hallar la distribución de los beneficios.
Encuentra el beneficio promedio, el intervalo de confianza del 95% y el riesgo de pérdidas. La pérdida es una ganancia negativa, calcúlala como una probabilidad y luego exprésala como un porcentaje.
Presenta tus conclusiones: propón una región para el desarrollo de pozos petrolíferos y justifica tu elección. ¿Coincide tu elección con la elección anterior en el punto 4.3?

In [23]:
def bootstrap_analysis(predicciones, n_samples=1000, n_pozos=200, precio_unidad=4500, presupuesto=100000000):
    beneficios = []
    for _ in range(n_samples):
        # Tomar muestra aleatoria con reemplazo
        muestra = np.random.choice(predicciones, size=500, replace=True)
        # Seleccionar los 200 mejores pozos
        mejores_pozos = np.sort(muestra)[-n_pozos:]
        # Calcular beneficio
        beneficio = np.sum(mejores_pozos) * precio_unidad - presupuesto
        beneficios.append(beneficio)
    
    beneficios = np.array(beneficios)
    
    # Calcular estadísticas
    media = np.mean(beneficios)
    intervalo = np.percentile(beneficios, [2.5, 97.5])
    riesgo = np.mean(beneficios < 0)
    
    return {
        'media': media,
        'intervalo': intervalo,
        'riesgo': riesgo
    }

# Analizar cada región
resultados = []
for i, preds in enumerate([preds_0, preds_1, preds_2]):
    res = bootstrap_analysis(preds)
    resultados.append({
        'region': i,
        'media': res['media'],
        'riesgo': res['riesgo']
    })
    print(f"\nRegión {i}:")
    print(f"Beneficio promedio: ${res['media']:,.2f}")
    print(f"Intervalo de confianza 95%: [${res['intervalo'][0]:,.2f}, ${res['intervalo'][1]:,.2f}]")
    print(f"Riesgo de pérdidas: {res['riesgo']*100:.2f}%")

# Filtrar regiones por riesgo y seleccionar la mejor
regiones_validas = [r for r in resultados if r['riesgo'] < 0.025]
mejor = max(regiones_validas, key=lambda x: x['media'])

print(f"\nMejor región: {mejor['region']} con beneficio promedio de ${mejor['media']:,.2f}")


Región 0:
Beneficio promedio: $3,317,350.37
Intervalo de confianza 95%: [$1,128,321.61, $5,682,449.94]
Riesgo de pérdidas: 0.40%

Región 1:
Beneficio promedio: $4,444,126.09
Intervalo de confianza 95%: [$510,758.54, $8,374,254.79]
Riesgo de pérdidas: 1.00%

Región 2:
Beneficio promedio: $2,678,005.39
Intervalo de confianza 95%: [$591,261.71, $4,616,520.25]
Riesgo de pérdidas: 0.70%

Mejor región: 1 con beneficio promedio de $4,444,126.09


#### Análisis y Decisión
##### Región 1 ofrece:
- El beneficio promedio más alto → mejor rendimiento esperado.

- Pero también mayor dispersión (IC más amplio) y un riesgo de pérdidas mayor (1%).

##### Región 0 es más conservadora:

- Menor beneficio esperado, pero riesgo muy bajo (0.40%).

##### Región 2 tiene el peor rendimiento esperado y no destaca ni por bajo riesgo ni por alto retorno.

Aunque Región 0 tenía la mayor ganancia potencial total, el análisis estadístico muestra que Región 1 es la más prometedora en términos de retorno promedio, con una aceptable exposición al riesgo.